# Random Effects Estimation (Example I)

### Intro and objectives


### In this lab you will learn:
1. examples of random effects estimation
2. how to fit random effects models in Python


## What I hope you'll get out of this lab
* The feeling that you'll "know where to start" when you need to fit random effects models
* Worked Examples
* How to interpret the results obtained

In [ ]:
!pip install wooldridge
!pip install linearmodels
import wooldridge as woo
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import linearmodels as plm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Example. Has the Return to Education Changed over Time?

#### The data in WAGEPAN are from Vella and Verbeek (1998). Each of the 545 men in the sample worked in every year from 1980 through 1987. Some variables in the data set change over time: experience, marital status, and union status are the three important ones. Other variables do not change: race and education are the key examples. 

#### If we use fixed effects, we cannot include race, education, or experience in the equation as they would be eliminated in the transformed model. Random effects models do allow us to include them though



#### We assume a model of the following form:

$log(wage)=\beta_0+\beta_1*education+\beta_2*black+\beta_3*hisp+\beta_4*exper+\beta_5*exper^2+\beta_6*married+\beta_7*union+\delta_0*y_{1980}+…+\delta_8*y_{1987}$






In [ ]:
wagepan = woo.dataWoo('wagepan')
wagepan = wagepan.set_index(['nr', 'year'], drop=False)

In [ ]:
wagepan.head()

nr  year  agric  black  bus  construc  ent  exper  fin  hisp  ...  \
nr year                                                                ...   
13 1980  13  1980      0      0    1         0    0      1    0     0  ...   
   1981  13  1981      0      0    0         0    0      2    0     0  ...   
   1982  13  1982      0      0    1         0    0      3    0     0  ...   
   1983  13  1983      0      0    1         0    0      4    0     0  ...   
   1984  13  1984      0      0    0         0    0      5    0     0  ...   

         union     lwage  d81  d82  d83  d84  d85  d86  d87  expersq  
nr year                                                               
13 1980      0  1.197540    0    0    0    0    0    0    0        1  
   1981      1  1.853060    1    0    0    0    0    0    0        4  
   1982      0  1.344462    0    1    0    0    0    0    0        9  
   1983      0  1.433213    0    0    1    0    0    0    0       16  
   1984      0  1.568125    0    0    0    1    0    0    0       25  

[5 rows x 44 columns]

In [ ]:
wagepan.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 4360 entries, (13, 1980) to (12548, 1987)
Data columns (total 44 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   nr        4360 non-null   int64  
 1   year      4360 non-null   int64  
 2   agric     4360 non-null   int64  
 3   black     4360 non-null   int64  
 4   bus       4360 non-null   int64  
 5   construc  4360 non-null   int64  
 6   ent       4360 non-null   int64  
 7   exper     4360 non-null   int64  
 8   fin       4360 non-null   int64  
 9   hisp      4360 non-null   int64  
 10  poorhlth  4360 non-null   int64  
 11  hours     4360 non-null   int64  
 12  manuf     4360 non-null   int64  
 13  married   4360 non-null   int64  
 14  min       4360 non-null   int64  
 15  nrthcen   4360 non-null   int64  
 16  nrtheast  4360 non-null   int64  
 17  occ1      4360 non-null   int64  
 18  occ2      4360 non-null   int64  
 19  occ3      4360 non-null   int64  
 20  occ4      43

In [ ]:
wagepan[['nr','year','black','bus','construc','exper','hisp','union','lwage','d81','d82','d83','expersq','married']].head(20)

nr  year  black  bus  construc  exper  hisp  union     lwage  d81  \
nr year                                                                      
13 1980  13  1980      0    1         0      1     0      0  1.197540    0   
   1981  13  1981      0    0         0      2     0      1  1.853060    1   
   1982  13  1982      0    1         0      3     0      0  1.344462    0   
   1983  13  1983      0    1         0      4     0      0  1.433213    0   
   1984  13  1984      0    0         0      5     0      0  1.568125    0   
   1985  13  1985      0    1         0      6     0      0  1.699891    0   
   1986  13  1986      0    1         0      7     0      0 -0.720263    0   
   1987  13  1987      0    1         0      8     0      0  1.669188    0   
17 1980  17  1980      0    0         0      4     0      0  1.675962    0   
   1981  17  1981      0    0         0      5     0      0  1.518398    1   
   1982  17  1982      0    0         0      6     0      0  1.559191    0   
   1983  17  1983      0    0         0      7     0      0  1.725410    0   
   1984  17  1984      0    0         0      8     0      0  1.622022    0   
   1985  17  1985      0    0         1      9     0      0  1.608588    0   
   1986  17  1986      0    0         1     10     0      0  1.572385    0   
   1987  17  1987      0    0         1     11     0      0  1.820334    0   
18 1980  18  1980      0    0         0      4     0      0  1.515963    0   
   1981  18  1981      0    0         0      5     0      0  1.735379    1   
   1982  18  1982      0    0         0      6     0      0  1.631744    0   
   1983  18  1983      0    0         0      7     0      0  1.998229    0   

         d82  d83  expersq  married  
nr year                              
13 1980    0    0        1        0  
   1981    0    0        4        0  
   1982    1    0        9        0  
   1983    0    1       16        0  
   1984    0    0       25        0  
   1985    0    0       36        0  
   1986    0    0       49        0  
   1987    0    0       64        0  
17 1980    0    0       16        0  
   1981    0    0       25        0  
   1982    1    0       36        0  
   1983    0    1       49        0  
   1984    0    0       64        0  
   1985    0    0       81        0  
   1986    0    0      100        0  
   1987    0    0      121        0  
18 1980    0    0       16        1  
   1981    0    0       25        1  
   1982    1    0       36        1  
   1983    0    1       49        1

In [ ]:
# RE model estimation:
reg= plm.RandomEffects.from_formula(
    formula='lwage ~ educ + black + hisp + exper + I(exper**2) +'
            'married + union + C(year)', data=wagepan)

results = reg.fit()

In [ ]:
# print regression table:
table = pd.DataFrame({'b': round(results.params, 4),
                      'se': round(results.std_errors, 4),
                      't': round(results.tstats, 4),
                      'pval': round(results.pvalues, 4)})
print(f'table: \n{table}\n')

table: 
                      b      se       t    pval
C(year)[T.1980]  0.0234  0.1514  0.1546  0.8771
C(year)[T.1981]  0.0638  0.1601  0.3988  0.6901
C(year)[T.1982]  0.0543  0.1690  0.3211  0.7481
C(year)[T.1983]  0.0436  0.1780  0.2450  0.8065
C(year)[T.1984]  0.0664  0.1871  0.3551  0.7225
C(year)[T.1985]  0.0811  0.1961  0.4136  0.6792
C(year)[T.1986]  0.1152  0.2052  0.5617  0.5744
C(year)[T.1987]  0.1583  0.2143  0.7386  0.4602
I(exper**2)     -0.0047  0.0007 -6.8623  0.0000
black           -0.1394  0.0480 -2.9054  0.0037
educ             0.0919  0.0107  8.5744  0.0000
exper            0.1058  0.0154  6.8706  0.0000
hisp             0.0217  0.0428  0.5078  0.6116
married          0.0638  0.0168  3.8035  0.0001
union            0.1059  0.0179  5.9289  0.0000



## How do we interpret the results?

#### Based on the fitted model, we conclude:

#### 1. Education, Hisp, Black, Experience, Married and Union are statistically significant.

#### 1. Married men earn 6.3% more than their single counterparts

#### 2. Union members earn 10.5% more than non-members.

#### 3. Each year of Education adds up 9.1% to the final salary.

#### 4. Each year of Experience adds up 10.5% to the final salary, yet this effect is dimishing (quadratic effect: -0.047%)

#### 5. People of hispanic ethinicity earn 2.7% more than the base ethicity (white).
#### 6. People of black ethnicity earn -13.9% less than the base ethnicity (white)